### Just change the code to predict oof feature individually for each type 

In [1]:
import numpy as np
import pandas as pd
import feather
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
import gc

import sys
sys.path.append("./../")
from utils import reduce_mem_usage

In [2]:
#実際予測時にはその化合物のデータは使えないので、厳密にはGroupKfoldで化合物ごとに分けるべき

In [3]:
features = [
        "Atom",
        "AtomPosition",
        "AtomDistance",
        "CouplingType",
        "AtomEnvironment",
        "AtomNeighbors",
        "BruteForce",
        "DistanceFromClosest",
        "ElectroNegFromClosest",
        "ACSF",
        "MullikenChargePred",
        "OpenBabelBasic",
        "CosineDistance",
        "PotentialPred", 
        "DisIsAllYouNeed"]

In [4]:
X_train = pd.concat([feather.read_dataframe("../features/" + feature + "_train.feather") for feature in features], axis=1)
X_test = pd.concat([feather.read_dataframe("../features/" + feature + "_test.feather") for feature in features], axis=1)

In [5]:
X_train.shape, X_test.shape

((4658147, 524), (2505542, 524))

In [6]:
X_train.head()

,atom_0,atom_1,x_0,x_1,y_0,y_1,z_0,z_1,atom_distance,dist_inv_0_sum,...,d_7_2,d_7_3,d_8_0,d_8_1,d_8_2,d_8_3,d_9_0,d_9_1,d_9_2,d_9_3
0,H,C,0.002150,-0.012698,-0.006031,1.085804,0.001976,0.008001,1.091953,1.297179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H,H,0.002150,1.011731,-0.006031,1.463751,0.001976,0.000277,1.783120,1.297179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H,H,0.002150,-0.540815,-0.006031,1.447527,0.001976,-0.876644,1.783147,1.297179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H,H,0.002150,-0.523814,-0.006031,1.437933,0.001976,0.906397,1.783157,1.297179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H,C,1.011731,-0.012698,1.463751,1.085804,0.000277,0.008001,1.091952,1.120798,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
contri = feather.read_dataframe("../data/input/scalar_coupling_contributions.feather")

In [8]:
contri.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [9]:
# keep nan as is
#coupling_types = X_train['type']
categorical_cols = list(X_train.columns[X_train.dtypes == object])


for col in categorical_cols:    
    print(f'Starting {col}')
    uniques = list(X_train[col].unique())
    if None in uniques:
        uniques.remove(None)
    mapping = dict(zip(uniques, range(1, len(uniques)+1)))
    X_train[col] = X_train[col].map(mapping)
    X_test[col] = X_test[col].map(mapping)

Starting atom_0
Starting atom_1
Starting type
Starting a1_is_aromatic
Starting a1_in_ring
Starting a1_in_ring3
Starting a1_in_ring4
Starting a1_in_ring5
Starting a1_in_ring6
Starting a1_in_ring7
Starting a1_in_ring8
Starting bond_atom
Starting tertiary_atom_0
Starting tertiary_atom_1
Starting tertiary_atom_2
Starting tertiary_atom_3
Starting tertiary_atom_4
Starting tertiary_atom_5
Starting tertiary_atom_6
Starting tertiary_atom_7
Starting tertiary_atom_8
Starting tertiary_atom_9


In [10]:
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)

Memory usage of dataframe is 16880.95 MB
Memory usage after optimization is: 9106.83 MB
Decreased by 46.1%
Memory usage of dataframe is 9070.43 MB
Memory usage after optimization is: 4881.69 MB
Decreased by 46.2%


In [11]:
gc.collect()

0

In [12]:
train_types = X_train['type'].values
test_types = X_test['type'].values

In [13]:
oof_train = pd.DataFrame()
oof_test = pd.DataFrame()


for col in ['fc', 'sd', 'pso', 'dso']:
    
    target = contri[col]
    
    oof_type = np.zeros(len(X_train))
    pred_type = np.zeros(len(X_test))
    
    for cur_type in np.unique(train_types):
        cur_type_idx_train = (train_types == cur_type)
        cur_type_idx_test = (test_types == cur_type)

        cur_type_train = X_train.iloc[cur_type_idx_train]
        cur_type_test = X_test.iloc[cur_type_idx_test]
        cur_type_target = target.iloc[cur_type_idx_train]
        cur_type_mols = contri['molecule_name'].iloc[cur_type_idx_train]
        
    
        kf = GroupKFold(n_splits=2)
    
        oof = np.zeros(len(cur_type_train))
        pred = np.zeros(len(cur_type_test))
        
        
        SEED = 42
        NUM_ROUNDS = 10000
        
        params = {
                    "num_leaves": 100,
                    "min_data_in_leaf": 100,
                    "objective": "regression",
                    "max_depth": 10,
                    "learning_rate": 0.2,
                    "boosting_type": "gbdt",
                    "subsample_freq": 1,
                    "subsample": 0.9,
                    "metric": "mae",
                    "reg_alpha": 0.1,
                    "reg_lambda": 0.3, 
                    "colsample_bytree": 0.9
                    }

        for train_idx, val_idx in kf.split(cur_type_train, groups=cur_type_mols):
            train_data = lgb.Dataset(cur_type_train.iloc[train_idx], label=cur_type_target.iloc[train_idx], categorical_feature=categorical_cols)
            val_data = lgb.Dataset(cur_type_train.iloc[val_idx], label=cur_type_target.iloc[val_idx], categorical_feature=categorical_cols)
            clf = lgb.train(params, train_data, NUM_ROUNDS, valid_sets=[train_data, val_data],
                            verbose_eval=1000, early_stopping_rounds=100)
            oof[val_idx] = clf.predict(cur_type_train.iloc[val_idx], num_iteration=clf.best_iteration)
            pred += clf.predict(cur_type_test, num_iteration=clf.best_iteration) / kf.n_splits

        oof_type[cur_type_idx_train] = oof
        pred_type[cur_type_idx_test] = pred
        
        print(f'{cur_type} for {col} done!')
        
    oof_train[col] = oof_type
    oof_test[col] = pred_type

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.392355	valid_1's l1: 0.759727
[2000]	training's l1: 0.224528	valid_1's l1: 0.725297
[3000]	training's l1: 0.140058	valid_1's l1: 0.714075
[4000]	training's l1: 0.0915718	valid_1's l1: 0.709096
[5000]	training's l1: 0.0613607	valid_1's l1: 0.706725
[6000]	training's l1: 0.041934	valid_1's l1: 0.705457
[7000]	training's l1: 0.0291264	valid_1's l1: 0.704716
[8000]	training's l1: 0.0205549	valid_1's l1: 0.704286
[9000]	training's l1: 0.0147301	valid_1's l1: 0.704021
[10000]	training's l1: 0.0106838	valid_1's l1: 0.703871
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.0106838	valid_1's l1: 0.703871
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.392988	valid_1's l1: 0.760302
[2000]	training's l1: 0.224601	valid_1's l1: 0.725826
[3000]	training's l1: 0.140311	valid_1's l1: 0.714317
[4000]	training's l1: 0.0914749	valid_1's l1: 0.709313
[5000]

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.058905	valid_1's l1: 0.179152
[2000]	training's l1: 0.0254415	valid_1's l1: 0.175082
[3000]	training's l1: 0.0124945	valid_1's l1: 0.174233
[4000]	training's l1: 0.00666873	valid_1's l1: 0.173943
[5000]	training's l1: 0.00385824	valid_1's l1: 0.173832
[6000]	training's l1: 0.00241745	valid_1's l1: 0.173781
[7000]	training's l1: 0.00163	valid_1's l1: 0.173753
[8000]	training's l1: 0.0011739	valid_1's l1: 0.17374
[9000]	training's l1: 0.00089551	valid_1's l1: 0.173732
[10000]	training's l1: 0.000722482	valid_1's l1: 0.173727
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.000722482	valid_1's l1: 0.173727
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0590071	valid_1's l1: 0.176245
[2000]	training's l1: 0.0257638	valid_1's l1: 0.172324
[3000]	training's l1: 0.0127042	valid_1's l1: 0.171463
[4000]	training's l1: 0.00677611	valid_1's l1: 0.1

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


...

Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00873696	valid_1's l1: 0.0175194
[2000]	training's l1: 0.00505537	valid_1's l1: 0.0169767
[3000]	training's l1: 0.00327045	valid_1's l1: 0.0168253
[4000]	training's l1: 0.00226282	valid_1's l1: 0.0167667
[5000]	training's l1: 0.00164986	valid_1's l1: 0.0167353
[6000]	training's l1: 0.00125591	valid_1's l1: 0.0167195
[7000]	training's l1: 0.000992151	valid_1's l1: 0.0167111
[8000]	training's l1: 0.000810777	valid_1's l1: 0.016706
[9000]	training's l1: 0.000682379	valid_1's l1: 0.0167031
[10000]	training's l1: 0.000592892	valid_1's l1: 0.0167007
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.000592892	valid_1's l1: 0.0167007
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00886493	valid_1's l1: 0.0178989
[2000]	training's l1: 0.00511511	valid_1's l1: 0.017332
[3000]	training's l1: 0.00329345	valid_1's l1: 0.0171804
[4000]	training's l1: 0

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00465732	valid_1's l1: 0.0063532
[2000]	training's l1: 0.00339256	valid_1's l1: 0.00592575
[3000]	training's l1: 0.00266304	valid_1's l1: 0.00576371
[4000]	training's l1: 0.00216638	valid_1's l1: 0.00567583
[5000]	training's l1: 0.00180807	valid_1's l1: 0.00563248
[6000]	training's l1: 0.00153341	valid_1's l1: 0.00560154
[7000]	training's l1: 0.00131824	valid_1's l1: 0.00558118
[8000]	training's l1: 0.00114691	valid_1's l1: 0.00556675
[9000]	training's l1: 0.00100828	valid_1's l1: 0.00555638
[10000]	training's l1: 0.000894348	valid_1's l1: 0.00554806
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.000894348	valid_1's l1: 0.00554806
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00468534	valid_1's l1: 0.00637373
[2000]	training's l1: 0.00341213	valid_1's l1: 0.00595095
[3000]	training's l1: 0.0026763	valid_1's l1: 0.005781
[4000]	trainin

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00069678	valid_1's l1: 0.00226568
[2000]	training's l1: 0.000534306	valid_1's l1: 0.00224381
[3000]	training's l1: 0.000479873	valid_1's l1: 0.00223711
Early stopping, best iteration is:
[3479]	training's l1: 0.000464216	valid_1's l1: 0.00223516
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.000693286	valid_1's l1: 0.00227561
[2000]	training's l1: 0.000532638	valid_1's l1: 0.0022542
[3000]	training's l1: 0.000479084	valid_1's l1: 0.00224751
Early stopping, best iteration is:
[3061]	training's l1: 0.000476841	valid_1's l1: 0.00224702
8 for dso done!


In [14]:
oof_train.shape, oof_test.shape

((4658147, 4), (2505542, 4))

In [15]:
oof_train.isnull().sum()

fc     0
sd     0
pso    0
dso    0
dtype: int64

In [16]:
oof_train.to_feather('../features/ScalarCouplingContributionsOof5_train.feather')
oof_test.to_feather('../features/ScalarCouplingContributionsOof5_test.feather')